In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral7bs500/checkpoint-175")

In [14]:
eval_prompt = """Generate 5 question from this text in array format to get most info about the text.
RESEARCH ARTICLE Open Access© The Author(s) 2024. Open Access   This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use,  sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and  the source, provide a link to the Creative Commons licence, and indicate if changes were made. The images or other third party material in this  article are included in the article’s Creative Commons licence, unless indicated otherwise in a credit line to the material. If material is not included  in the article’s Creative Commons licence and your intended use is not permitted by statutory regulation or exceeds the permitted use, you will  need to obtain permission directly from the copyright holder. To view a copy of this licence, visit http://creativecommons.org/licenses/by/4.0/ . The  Creative Commons Public Domain Dedication waiver ( http://creativecommons.org/publicdomain/zero/1.0/ ) applies to the data made available  in this article, unless otherwise stated in a credit line to the data.Mateo-Otero et al. Biological Research             (2024) 57:5  https://doi.org/10.1186/s40659-024-00483-4 ATP by oxidative phosphorylation (OXPHOS) [ 3], has  been found to positively influence oocyte fertilisation in  humans [ 4–6]. In spite of this, a metabolomics approach  carried out using the pig as an animal model found that  metabolites linked to OXPHOS were related to subop - timal in vitro fertilisation (IVF) outcomes in terms of  altered embryo development [ 2]. While the underlying  mechanisms of these associations are yet to be uncov - ered, one could reasonably suggest that reactive oxygen  species (ROS) are implicated. Sperm mitochondria have been proposed as a major  source of ROS [ 7], produced from the leakage of respira - tory chain electrons [ 8]. Under physiological conditions,  ROS are involved in the regulation of sperm function,  participating in the modulation of sperm capacita - tion and acrosomal reaction and, even, oocyte fertilisa - tion [ 9]. Overproduction of ROS, nonetheless, results in Background The energetic metabolic signature of mammalian sperm  influences reproductive outcomes, both in terms of  oocyte fertilisation and embryo development [ 1, 2].  Indeed, mitochondrial membrane potential (MMP),  which indirectly reflects the cellular capacity to produce Biological Research †Yentel Mateo-Otero and Marc Llavanera contributed equally to this  work. *Correspondence: Yentel Mateo-Otero yentel.mateo@udg.edu 1Biotechnology of Animal and Human Reproduction (TechnoSperm),  Institute of Food and Agricultural Technology, University of Girona,   Girona ES-17003, Spain 2Unit of Cell Biology, Department of Biology, Faculty of Sciences,  University of Girona, Girona ES- 17003, Spain 3Catalan Institution for Research and Advanced Studies (ICREA),   Barcelona ES-08010, SpainAbstract Background  Basal energetic metabolism in sperm, particularly oxidative phosphorylation, is known to condition  not only their oocyte fertilising ability, but also the subsequent embryo development. While the molecular pathways  underlying these events still need to be elucidated, reactive oxygen species (ROS) could have a relevant role. We,  therefore, aimed to describe the mechanisms through which mitochondrial activity can influence the first stages of  embryo development. Results  We first show that embryo development is tightly influenced by both intracellular ROS and mitochondrial  activity. In addition, we depict that the inhibition of mitochondrial activity dramatically decreases intracellular ROS  levels. Finally, we also demonstrate that the inhibition of mitochondrial respiration positively influences sperm DNA  integrity, most likely because of the depletion of intracellular ROS formation. Conclusion  Collectively, the data presented in this work reveals that impairment of early embryo development may  result from the accumulation of sperm DNA damage caused by mitochondrial-derived ROS. Keywords  Sperm metabolism, Mitochondrial respiration, electron transport chain, Cyanide, Oocyte fertilisation,  Embryo development, Oxidative phosphorylationEmbryo development is impaired by sperm  mitochondrial-derived ROS Yentel Mateo-Otero1,2*†, Marc Llavanera1,2†, Marc Torres-Garrido1,2 and Marc Yeste1,2,3\n\n"""
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")
ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=500, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generate 5 question from this text in array format to get most info about the text.
RESEARCH ARTICLE Open Access© The Author(s) 2024. Open Access   This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use,  sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and  the source, provide a link to the Creative Commons licence, and indicate if changes were made. The images or other third party material in this  article are included in the article’s Creative Commons licence, unless indicated otherwise in a credit line to the material. If material is not included  in the article’s Creative Commons licence and your intended use is not permitted by statutory regulation or exceeds the permitted use, you will  need to obtain permission directly from the copyright holder. To view a copy of this licence, visit http://creativecommons.org/licenses/by/4.0/ . The  Cr